Imports

In [59]:
import os
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

Load Stacked Dataset

In [60]:
data_path = "../data/processed/stacked_dataset.parquet"
df = pd.read_parquet(data_path)

print("Loaded dataset:", df.shape)

Loaded dataset: (35954029, 12)


Confusion Matrix Plot Function

In [61]:
def plot_confusion(cm, classes, title, save_path):
    plt.figure(figsize=(6,5))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], "d"),
                     ha="center", va="center")

    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()

    plt.savefig(save_path)
    plt.close()

Train/Val/Test Split

In [62]:
# Fire-wise Split
fires = df["fire_name"].unique()
fires_train, fires_temp = train_test_split(
    fires, test_size=0.3, random_state=42
)
fires_val, fires_test = train_test_split(
    fires_temp, test_size=0.5, random_state=42
)

print("Train fires:", fires_train)
print("Val fires:", fires_val)
print("Test fires:", fires_test)

df_train_full = df[df["fire_name"].isin(fires_train)]
df_val_full   = df[df["fire_name"].isin(fires_val)]
df_test_full  = df[df["fire_name"].isin(fires_test)]

# Stratified subsampling per split
def stratified_subsample(df_split, target_samples):
    """Stratified subsampling inside a single fire-group split."""
    target_samples = min(target_samples, len(df_split))
    
    sss = StratifiedShuffleSplit(
        n_splits=1,
        train_size=target_samples,
        random_state=42
    )
    idx, _ = next(sss.split(df_split, df_split["severity"]))
    return df_split.iloc[idx].reset_index(drop=True)

df_train = stratified_subsample(df_train_full, 2_000_000)
df_val   = stratified_subsample(df_val_full,   300_000)
df_test  = stratified_subsample(df_test_full,  300_000)

print("Train:", len(df_train))
print("Val:", len(df_val))
print("Test:", len(df_test))

# Final Splits
X_train = df_train.drop(columns=["severity", "fire_name", "dnbr", "dndvi", "nbr_post", "ndvi_post"])
y_train = df_train["severity"]

X_val = df_val.drop(columns=["severity", "fire_name", "dnbr", "dndvi", "nbr_post", "ndvi_post"])
y_val = df_val["severity"]

X_test = df_test.drop(columns=["severity", "fire_name", "dnbr", "dndvi", "nbr_post", "ndvi_post"])
y_test = df_test["severity"]

Train fires: ['bootleg_fire' 'zogg_fire' 'camp_fire' 'dixie_fire' 'carr_fire'
 'glass_fire']
Val fires: ['east_troublesome_fire']
Test fires: ['red_salmon_fire' 'caldor_fire']
Train: 2000000
Val: 300000
Test: 300000


Model Definitions

In [63]:
models = {
    "logistic_regression": LogisticRegression(
        max_iter=2000,
        class_weight="balanced"
    ),
    "random_forest": RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        n_jobs=-1,
        class_weight="balanced"
    ),
    "xgboost": XGBClassifier(
        n_estimators=400,
        max_depth=10,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        eval_metric="mlogloss"
    ),
    "mlp": MLPClassifier(
        hidden_layer_sizes=(128, 64),
        activation="relu",
        max_iter=50
    )
}

os.makedirs("../data/models", exist_ok=True)
os.makedirs("../figures", exist_ok=True)

Train and Evaluate Models

In [64]:
results = []

for name, model in models.items():
    print(f"Training {name}")

    # Train
    model.fit(X_train, y_train)

    # Validation prediction
    val_pred = model.predict(X_val)
    test_pred = model.predict(X_test)

    # Validation metrics
    val_f1 = f1_score(y_val, val_pred, average="weighted")
    test_f1 = f1_score(y_test, test_pred, average="weighted")

    print(f"{name} validation F1:", val_f1)
    print(f"{name} test F1:", test_f1)

    # Save model
    model_path = f"../data/models/{name}.pkl"
    joblib.dump(model, model_path)
    print(f"Saved model: {model_path}")

    # Confusion matrix
    cm_val = confusion_matrix(y_val, val_pred)
    cm_test = confusion_matrix(y_test, test_pred)

    plot_confusion(
        cm_val,
        classes=np.unique(y_train),
        title=f"{name} - Validation Confusion Matrix",
        save_path=f"../figures/{name}_val_cm.png"
    )

    plot_confusion(
        cm_test,
        classes=np.unique(y_train),
        title=f"{name} - Test Confusion Matrix",
        save_path=f"../figures/{name}_test_cm.png"
    )

    results.append({
        "model": name,
        "val_f1": val_f1,
        "test_f1": test_f1
    })

    # Print classification report
    print("\nClassification Report (Validation):")
    print(classification_report(y_val, val_pred))

    print("\nClassification Report (Test):")
    print(classification_report(y_test, test_pred))


Training logistic_regression


c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


logistic_regression validation F1: 0.14413043386573887
logistic_regression test F1: 0.24524262351878395
Saved model: ../data/models/logistic_regression.pkl

Classification Report (Validation):
              precision    recall  f1-score   support

           0       0.91      0.08      0.15    280527
           1       0.05      0.52      0.09     13031
           2       0.02      0.44      0.04      6419
           3       0.00      0.04      0.00        23

    accuracy                           0.11    300000
   macro avg       0.25      0.27      0.07    300000
weighted avg       0.85      0.11      0.14    300000


Classification Report (Test):
              precision    recall  f1-score   support

           0       0.97      0.18      0.30    226908
           1       0.08      0.04      0.06     49834
           2       0.08      0.28      0.13     20408
           3       0.02      0.93      0.03      2850

    accuracy                           0.17    300000
   macro avg   

c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

              precision    recall  f1-score   support

           0       0.81      0.67      0.73    226908
           1       0.26      0.33      0.29     49834
           2       0.06      0.14      0.08     20408
           3       0.00      0.00      0.00      2850

    accuracy                           0.57    300000
   macro avg       0.28      0.29      0.28    300000
weighted avg       0.66      0.57      0.61    300000

Training xgboost
xgboost validation F1: 0.9035075164741168
xgboost test F1: 0.6117457942608233
Saved model: ../data/models/xgboost.pkl

Classification Report (Validation):
              precision    recall  f1-score   support

           0       0.94      1.00      0.97    280527
           1       0.19      0.00      0.00     13031
           2       0.00      0.00      0.00      6419
           3       0.00      0.00      0.00        23

    accuracy                           0.93    300000
   macro avg       0.28      0.25      0.24    300000
weighted avg 

c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

mlp validation F1: 0.8841459027315752
mlp test F1: 0.5822081573116358
Saved model: ../data/models/mlp.pkl

Classification Report (Validation):
              precision    recall  f1-score   support

           0       0.93      0.95      0.94    280527
           1       0.03      0.03      0.03     13031
           2       0.13      0.00      0.01      6419
           3       0.00      0.00      0.00        23

    accuracy                           0.89    300000
   macro avg       0.27      0.25      0.25    300000
weighted avg       0.88      0.89      0.88    300000


Classification Report (Test):
              precision    recall  f1-score   support

           0       0.81      0.63      0.71    226908
           1       0.22      0.23      0.22     49834
           2       0.08      0.29      0.13     20408
           3       0.19      0.00      0.01      2850

    accuracy                           0.53    300000
   macro avg       0.33      0.29      0.27    300000
weighted av

c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\John Waugh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

Save comparison table

In [65]:
results_df = pd.DataFrame(results)
results_df.to_csv("../data/models/metrics.csv", index=False)
print("\nSaved metrics table to ../data/models/metrics.csv")
print(results_df)


Saved metrics table to ../data/models/metrics.csv
                 model    val_f1   test_f1
0  logistic_regression  0.144130  0.245243
1        random_forest  0.903705  0.608999
2              xgboost  0.903508  0.611746
3                  mlp  0.884146  0.582208
